# 二维机翼模拟（浸入边界方法）


AERO-Suite里的**AERO-F**是流体仿真模拟器，这里是详细的[**AERO-F**教程](https://frg.bitbucket.io/aero-f/index.html)。

二维机翼模拟是一个纯流体仿真。本次仿真模拟的文件都在`NACA/ALE`文件夹中，
在这个文件中一般有3个子文件夹：`sources`, `simulations`和`data`。
- `sources`文件夹一般存放网格相关的源文件
- `simulations`文件夹一般存放仿真模拟的输入文件
- `data`文件夹一般存放仿真模拟过程中生成的一些文件。

对于AERO-F, 一般流体仿真文件夹示意图如下：

<img src="../../../../Figs/Folders.png" width="500" />


以下是本次仿真模拟的详细步骤。
在开始本次仿真模拟前，我们需要安装相关的软件，请参加安装教程,比如[如何在北京大学未名一号上安装AERO-Suite](../../../../Install/Install_PKU.ipynb)。
如果我们之前已经安装好相关软件，我们可以读取 [.bashrc_frg](../../../../Install/bashrc_frg) 刷新当前shell环境，载入各个软件的命令
```shell
source ~/.bashrc_frg 
```


## 生成网格



Gmsh使用后缀为'.geo'的文件。
在我们的流体网格 'sources/domain.geo' 文件里，我们首先选取了方形的二维计算区域（Background mesh）的坐标点，进行连线构成线、面，然后在第三维上延拓（Extrude）几层，最后标记边界名称，包括方形计算区域的远场边界条件OutletFixedSurface （由于我们对远场边界条件用Steger-Warming格式，该方法自动区别流入流出边界条件，这里也可以用InletFixedSurface)和第三维两个面的边界条件SymmetryFixedSurface。
在我们的固体网格 'sources/airfoil.top' 文件里，我们首先选取了表示机翼形状的坐标点，进行连线构成曲线，然后在第三维上延拓（Extrude）几层构成曲面，最后标记边界名称，为StickMovingSurface （这里也可以用StickFixedSurface，对于StickFixedSurface，在流固耦合计算时，我们会忽略该边界上的固体的受力）。
值得一提的是对于二维模拟，一般我们的固体网格会选取得稍微宽一些。因为对于浸入边界方法，我们从远场边界选取一个点，认为这个点在流体里面，然后对于和这个点有边相连的点，如果这条边和固体网格不相交，我们就把这个点标注为流体里的点，由于判断一条边是否和固体网格相交，需要一定的容错，所以我们一般会把固体网格选取得稍微宽一些。







由于AERO-F只支持三维四面体网格的流体仿真，我们使用的网格也是三维网格。
任意二维问题都可以转化成三维的问题，即在第三维使用少许几层网格（比如一层），在第三维两个面上作用对称边界条件。

本次仿真，我们使用 [**Gmsh**](http://gmsh.info) 来生成网格。**Gmsh**的生成网格的基本方式是，
首先输入点的坐标（Point），用这些点连成线 (Line）或曲线（比如 Spline），再由线组成面(比如 Plane Surface)，最后由面构成体积。
我们可以对面和体积进行打网格，对于不同的面，**Gmsh**也需要标注不同的边界名称 (Physical Surface)，用于**AERO-F**识别不同的边界条件。**Gmsh**同时支持用户界面以及脚本文件。

**Gmsh**脚本文件使用后缀为`.geo`的文件。本次仿真模拟的`.geo`文件是`sources`文件夹里的 [fluid.geo](../../sources/fluid.geo) 和[structure.geo](../../sources/structure.geo)文件。

[fluid.geo](../../sources/fluid.geo)是流体网格文件，我们首先选取了方形的二维计算区域的坐标点，
进行连线构成面，然后在第三维上延拓（Extrude）三层，最后标记边界名称。
这些边界包括：方形计算区域的远场边界条件 OutletFixedSurface （由于我们对远场边界条件的处理使用的是Steger-Warming格式，
该方法自动区别流入流出边界条件，所以这里也可以用InletFixedSurface），第三维两个面的边界条件 SymmetryFixedSurface。
值得注意的是我们也在流体网格文件中标注了固体的位置，用于加密网格进而生成高质量的网格。

[structure.geo](../../sources/fluid.geo)是机翼网格文件，我们首先选取了机翼的坐标点，进行连线构成面，然后在第三维上延拓（Extrude）一层，最后标记边界名称为。 StickMovingSurface （这里也可以用StickFixedSurface，
对于StickFixedSurface，在流固耦合计算时，我们会忽略该边界上的固体的受力）。
值得一提的是对于二维模拟，一般我们的固体网格会选取得稍微宽一些。因为对于浸入边界方法，我们从远场边界选取一个点，认为这个点在流体里面，然后对于和这个点有边相连的点，如果这条边和固体网格不相交，我们就把这个点标注为流体里的点，由于判断一条边是否和固体网格相交，需要一定的容错，所以我们一般会把固体网格选取得稍微宽一些。



用**Gmsh**打开 [fluid.geo](../../sources/fluid.geo)，生成3D流体网格 [fluid.msh](../../sources/fluid.msh) 文件；打开[structure.geo](../../sources/structure.geo)，生成2D固体网格 [structure.msh](../../sources/structure.msh)。 这也可以在`sources`文件夹中用命令行完成

```shell
gmsh -3 fluid.geo -format msh22
gmsh -2 structure.geo -format msh22
```

值得注意的是，因为**gmsh2top**的输入是MSH 2.2 格式，因此我们需要在命令行里使用`-format msh22`。如果是用图形化界面储存，需要在Export成Gmsh MSH格式的时候选择 Version 2 ASCII 的 Format。


**AERO-F**使用后缀为`.top`的文件，`.top`文件是更简洁的网格存储方式。`.top`文件里，首先是顶点（Nodes），然后是元（Elements），这些元包括四面体（代号为5）和三角形(代号为4)。
我们可以用**gmsh2top**软件，把`.gmsh`文件转化为`.top`文件，这可以在`sources`文件夹中用以下命令行完成

```shell
gmsh2top fluid
gmsh2top structure
```

这里我们展示了`fluid.geo`和生成的网格:
<img src="../../../../Figs/NACA_EBM.png" width="500" />


# 网格分区

对于流体仿真，很多时候需要使用超算进行并行，我们需要对网格进行分区。我们使用**partnmesh**软件，能把流体网格划分为任意多个区进行并行。本次仿真模拟我们把网格划分成4个区域，使用4个核进行运算。这可以在`ALE`文件里用以下命令行完成

```shell
partnmesh sources/fluid.top 4
```

这会在`sources`文件夹中，生成有分区标注的`fluid.top.dec.4`文件。


之后我们需要用**sower**去生成各个区域网格的详细信息，把这些信息文件储存在`data`文件夹中，这可以在`ALE`文件夹里用以下命令行完成

```shell
sower -fluid -mesh sources/fluid.top -dec sources/fluid.top.dec.4 -cpu 4 -cluster 4 -output data/OUTPUT
```

这会在`data`文件夹中生成以`OUTPUT`为前缀的文件，包括各区域的网格信息比如`OUTPUT.msh1`，标注信息比如`OUTPUT.dec1`，各4个文件，以及CPU和分区的映射`OUTPUT.4cpu`和各个分区的连接信息`OUTPUT.con`。

## 流体计算

流体计算的所有信息都在**AERO-F**的输入文件里。对于本次模拟，这个输入文件是 [simulations/case1/FluidFile.Steady](./FluidFile.Steady)。在这个文件夹里，我们一般有一些子文件夹：

- `references`用于储存模拟中途的流体状态，用于重启（Restart）模拟；
- `results`用于储存模拟中途的输出；
- `postpro`用于最后后处理过后的模拟结果。

**AERO-F**的输入文件分为许多个模块，值得一提的是，模块之间的顺序以及模块里指定的参数的顺序都没有任何限制。接下来，我们具体讲解本次仿真模拟的输入文件。


这里我们首先指定求解问题的类型，本次仿真使用浸入边界方法（Embedded），是无量纲的（NonDimensional）定常流（Steady）的模拟。

```shell
under Problem {
  Type = Steady;
  Mode = NonDimensional;
  Framework = Embedded;
}
```

然后指定网格相关的文件（Input），它们是网格分区预处理的生成文件，一般在`data`文件夹里面。这里可以给出这些网格相关文件的前缀，整体的前缀或者每一部分的前缀，**AERO-F**会自动去查找这些文件。值得注意的是我们需要指定固体网格（EmbeddedSurface），这里是从前缀指定的位置开始寻找。

```shell
under Input {
  Prefix = "../../data/";
  GeometryPrefix = "OUTPUT";
  EmbeddedSurface = "../sources/structure.top";
}
```

然后指定仿真模拟中间的输出。这里包括我们感兴趣的一些量（Postpro）和用于重启模拟的文件（Restart），其中频率（Frequency）指定每多少步输出一次，如果频率是0，只在最后模拟结束时输出。但我们要输出固体表面的量，比如压力系数（EmbeddedSurfacePressureCoefficient），我们需要用 PostprocessSides 指定输出的是哪一侧的量，其中 1 表示网格外法线一侧。固体网格可以用**

```shell
under Output {
  under Postpro {
    Frequency = 100;
    Prefix = "results.Steady/";
    Residual = "Residual.out";
    LiftandDrag = "../postpro.Steady/LiftandDrag.out";
    PostprocessSides = 1;
    EmbeddedSurfacePressureCoefficient = "../postpro.Steady/EmbeddedSurfacePressureCoefficient.xpost";
    Length = 1.0;
    Surface = 1.0;
    XM = 0.0;
    YM = 0.0;
    ZM = 0.0;
    Pressure = "Pressure.bin";
    Mach = "Mach.bin";
  }
  under Restart {
    Frequency = 0;
    Prefix = "references.Steady/";
    Solution = "Solution.bin";
    RestartData = "Restart.data";
    Position = "Position.bin";
    EmbeddedPosition = "EmbeddedPosition.dat";
  }
}
```

然后指定边界条件。这里包括远场边界条件（Inlet）和固体表面边界条件（Wall）。

```shell
under BoundaryConditions {
  under Inlet {
    Type = External;
    Mach = 0.8;
    Alpha = 0.0;
    Beta = 5.0;
  }
}
```

由于是无量纲模拟，我们也需要指定参考状态。

```shell
under ReferenceState {  
  Length = 1.0;
}
```

然后指定我们要求解的方程以及相关的参数，比如Euler方程或者Navier Stokes方程，以及湍流闭包模型。

```shell
under Equations {
  Type = Euler;
  under FluidModel[0] {
    Fluid = PerfectGas;
    under GasModel {
      SpecificHeatRatio = 1.4;
    }
  }
}
```

然后指定空间离散格式。

```shell
under Space {
  under NavierStokes {
    Flux = Roe;
    Reconstruction = Linear;
    AdvectiveOperator = FiniteVolume;
    Limiter = None;
    Gradient = LeastSquares;
    Dissipation = SecondOrder;
    Beta = 0.3333333333333333;
    Gamma = 1.0;
  }
  under Boundaries {
    Type = StegerWarming;
  }
}
```

然后指定时间离散格式，包括CFL条件，显、隐格式，牛顿迭代的参数等。

```shell
under Time {
  MaxIts = 500;
  Eps = 1.0E-6;
  under CflLaw {
    Strategy = Fixed;
    Cfl0 = 20.0;
  }
  CheckLinearSolver = Off;
  Type = Implicit;
  under Implicit {
    Type = BackwardEuler;
    MatrixVectorProduct = FiniteDifference;
    under Newton {
      MaxIts = 1;
      Eps = 0.01;
      FailSafe = Off;
      under LinearSolver {
        under NavierStokes {
          Type = Gmres;
          MaxIts = 50;
          KrylovVectors = 50;
          Eps = 0.01;
          //Output = "stderr";
          under Preconditioner {
            Type = Ras;
            Fill = 0;
          }
        }
      }
    }
  }
}
```

最后指定浸入边界方法的参数，包括求解半黎曼问题的位置（TypeHalfRiemannProblem），选择在控制体积边界（Surrogate）能避免外插更加稳定，但是只有一阶精度。 把流体速度投影到一维求解半黎曼问题使用的向量（RiemannNormal）等。

```shell
under EmbeddedFramework {
  TypeHalfRiemannProblem = Surrogate;
  RiemannNormal = Structure;  
}
```

我们可以在`simulations/case1`中，用以下命令行在4个核上运行**AERO-F**。

```shell
mpirun -N 4 aerof.opt FluidFile.Steady
```

我们感兴趣的一些量，比如流体的压强（比如Pressure.bin1）马赫数（Mach.bin1）和升力阻力（LiftandDrag）都存在 `results.Steady`文件夹, 像压强和马赫数这些流体状态的量，都是每一个分区上的数据，因此各有4个文件。
用于重启模拟的文件，都存在`references.Steady`文件夹。


## 结果后处理

由于**AERO-F**的并行，是每个CPU在各自分配到的划分的网格上进行计算，输出结果也是划分网格上的结果，我们需要首先用**sower**把输出结果拼接起来。我们可以在`simulations/case1`中，用以下命令行拼接关于压强和马赫数的结果。

```shell
sower -fluid -merge -con ../../data/OUTPUT.con -mesh ../../data/OUTPUT.msh \
	-result results.Steady/Mach.bin -output postpro.Steady/Mach
    
sower -fluid -merge -con ../../data/OUTPUT.con -mesh ../../data/OUTPUT.msh \
	-result results.Steady/Pressure.bin -output postpro.Steady/Pressure
```

这里的输入包括网格分区的信息，即在`data`文件夹中各个分区的连接信息`OUTPUT.con`和各区域的网格信息（比如`OUTPUT.msh1`），以及模拟输出在`results.Steady`文件夹中的信息（比如`Mach.bin`）。结果会存放在`postpro.Steady`文件夹里面，分别是 `Mach.xpost`和 `Pressure.xpost`。 

由于拼接起来的文件是后缀为`.xpost`的文件，为了使用**paraview**做后处理，我们需要用**xp2exo**把后缀为`.xpost`的文件转化为后缀为`.exo`的文件, 我们可以在`simulations/case1`中，用以下命令行进行转换

```shell
xp2exo ../../sources/fluid.top fluid_solution.exo \
                   ../../sources/fluid.top.dec.4 \
                   postpro.Steady/Mach.xpost postpro.Steady/Pressure.xpost            
```

这里的输入包括在`sources`文件夹中的流体网格`fluid.top`和流体分区的信息`fluid.top.dec.4`，以及**sower**拼接后的我们感兴趣的流体各个量（包括`Mach.xpost`和`Pressure.xpost`，最终结果会存放在`fluid_solution.exo`面。


我们可以用[**paraview**](https://www.paraview.org/tutorials/)读入`fluid_solution.exo`进行可视化。
这里我们展示了**paraview**界面和稳态时刻的马赫场：
<img src="../../../../Figs/NACA_ALE_Euler_Steady.png" width="500" />


## bash脚本运行


对于网格划分的前处理，我们可以把这些命令行写在`ALE`文件夹里的脚本文件里，比如[preprocess.sh](../../preprocess.sh)。我们可以用以下命令行运行脚本

```shell
bash preprocess.sh
```


对于运行**AERO-F**以及结果的后处理，我们可以这些命令行写在`simulations/case1`文件夹里的脚本文件里，比如`run.Steady.sh`。我们可以用以下命令运行脚本

```shell
bash run.Steady.sh
```

为了在集群上提交作业，我们需要用 [sbatch命令](https://hpc.pku.edu.cn/_book/guide/slurm/sbatch.html) 将作业提交到计算节点上执行，在`simulations/case1`文件夹里，我们可以编写作业脚本文件`NACA.EBM.Case1.Steady.sh`申请计算节点，进行计算。可以用以下命令行提交作业

```shell
sbatch NACA.EBM.Case1.Steady.sh
```



